# Jupyter to check splits genetic saved results

In [1]:
import sys
import os 
import random
sys.path.append("..")
from utils import read_patients_metadata
from config import *

import numpy as np
#IMPORTANT pip install numpy==1.16.1

Loading configuration...
Configuration loaded successfully!
_____________________________



In [2]:
TOTAL_FOLDS = 4
#PERCENTAGES = (3/10, 7/50, 7/50, 7/50, 7/50, 7/50)
PERCENTAGES = (0.25, 0.25, 0.25, 0.25)


In [27]:
def show_results(DATASET_ID, PATH):
    DATASET = AVAILABLE_DATASETS[DATASET_ID]
    
    metadata_original = read_patients_metadata(PATCH_SIZE)[DATASET_ID]

    volumes = [int(metadata_original[pat][les]["volume"]) 
               for db in metadata_original 
               for pat in metadata_original 
               for les in metadata_original[pat]]

    pos_to_subj = {}
    subj_to_pos = {}
    counter = 0

    CENTER_PATIENTS = []
    for pat in DATASET.get_subjects():
        pos_to_subj[counter] = pat
        subj_to_pos[pat] = counter
        CENTER_PATIENTS.append(counter)
        counter += 1

    metadata = {}
    for pat_int in list(metadata_original.keys()):
        pat = f"{pat_int:03d}"
        metadata[subj_to_pos[pat]] = metadata_original[pat_int]
        for les in list(metadata[subj_to_pos[pat]].keys()):
            if bool(metadata[subj_to_pos[pat]][les]["ignore"]):
                metadata[subj_to_pos[pat]].pop(les, None)
    
    PATS_COUNTS = np.zeros([len(subj_to_pos), 2], dtype='int32')

    for pat in CENTER_PATIENTS:
        patient_rims_count = len([les for les in metadata[pat].keys() if int(les) // 1000 == 1])
        patient_non_rims_count = len([les for les in metadata[pat].keys() if int(les) // 1000 == 2])
        PATS_COUNTS[pat][0] = patient_rims_count
        PATS_COUNTS[pat][1] = patient_non_rims_count

    print(f"{np.sum(PATS_COUNTS, axis=0)}")
    TOTAL_PATIENTS = PATS_COUNTS.shape[0]
    
    # Load results
    idv = np.load(PATH)
    
    # Print results
    pos_per_fold = []
    for i in range(len(idv[0])):
        print(f"\nFold {i}")
        positions = np.where(idv[0][i] == True)[0]
        patients = [pos_to_subj[p] for p in positions]
        print(tuple(patients))

        pos_per_fold.append(positions)
        
    results = []
    for i in range(len(pos_per_fold)):
        pos, neg = np.sum(PATS_COUNTS[pos_per_fold[i]], axis=0)
        results.append((pos, neg, pos+neg))

    results = pd.DataFrame(results, columns=["pos", "neg", "total"])
    
    for p in PERCENTAGES:
        print(f"{np.round(np.sum(PATS_COUNTS, axis=0) * p).astype(int)}")

    print(results)
    
    return results

In [34]:

PATH_BASEL = f"/Users/german/Desktop/RESULTS_FINAL_4/DATASET_BASEL/16_1.npy"
PATH_CHUV = f"/Users/german/Desktop/RESULTS_FINAL_4/DATASET_CHUV/225.npy"


In [29]:

results_basel = show_results(DATASET_BASEL_ID, PATH_BASEL)

[ 312 2716]

Fold 0
('063', '064', '065', '066', '067', '069', '075', '078', '080', '097', '099', '100', '108', '112', '114', '116', '120', '125', '131')

Fold 1
('057', '059', '060', '062', '072', '073', '081', '082', '092', '095', '098', '107', '110', '122', '124', '126', '129', '130', '132')

Fold 2
('068', '077', '079', '083', '084', '093', '096', '103', '109', '113', '115', '118')

Fold 3
('056', '058', '061', '070', '071', '074', '076', '085', '094', '101', '102', '104', '105', '106', '117', '121', '123', '127', '128')
[ 78 679]
[ 78 679]
[ 78 679]
[ 78 679]
   pos  neg  total
0   78  684    762
1   78  682    760
2   78  679    757
3   78  671    749


In [30]:

results_chuv = show_results(DATASET_CHUV_ID, PATH_CHUV)

[ 145 1971]

Fold 0
('011', '012', '016', '017', '021', '024', '028', '030', '032', '039', '041', '050', '053')

Fold 1
('003', '005', '010', '013', '014', '023', '027', '029', '044', '045', '047', '048', '052', '054', '055')

Fold 2
('001', '008', '015', '020', '022', '025', '031', '034', '035', '037', '040', '042', '043', '046')

Fold 3
('002', '004', '006', '007', '009', '018', '019', '026', '033', '036', '038', '049', '051')
[ 36 493]
[ 36 493]
[ 36 493]
[ 36 493]
   pos  neg  total
0   39  485    524
1   35  495    530
2   34  494    528
3   37  497    534


In [31]:
# 0-0
# 1-2
# 2-1
# 3-3

aux = results_chuv.iloc[1].copy()
results_chuv.iloc[1] = results_chuv.iloc[2]
results_chuv.iloc[2] = aux

In [32]:
results_chuv + results_basel

,pos,neg,total
0,117,1169,1286
1,112,1176,1288
2,113,1174,1287
3,115,1168,1283
